In [0]:
from pyspark.sql import Row
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.types import FloatType
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf
from pyspark.sql import functions as F 
from pyspark.sql.functions import explode, col, udf, mean as _mean, stddev as _stddev, log, log10
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.functions import lit

In [0]:
df_train = spark.read.csv('/FileStore/tables/train_features.csv', header='true',inferSchema = True)
df_target = spark.read.csv('/FileStore/tables/train_targets_scored.csv', header='true',inferSchema = True)

In [0]:
df_train.show(5)

+------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+------+------+-------+------+-------+-------+-------+-------+-------+-------+------+-------+------+-------+-------+-------+-------+-------+-------+------+-------+-------+------+------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+------+-------+-------+-------+-------+------+------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+------+-------+-------+-------+-------+-------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+-------+-------+-------+------+------+-------+-------+------+-------+-------+-------+-------+-------+------+------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+---

In [0]:
df_train.toPandas().head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,...,c-60,c-61,c-62,c-63,c-64,c-65,c-66,c-67,c-68,c-69,c-70,c-71,c-72,c-73,c-74,c-75,c-76,c-77,c-78,c-79,c-80,c-81,c-82,c-83,c-84,c-85,c-86,c-87,c-88,c-89,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,-0.0921,1.1830,0.1530,0.5574,-0.4015,0.1789,-0.6528,-0.7969,0.6342,0.1778,-0.3694,-0.5688,-1.1360,-1.1880,0.6940,0.4393,0.2664,0.1907,0.1628,-0.2853,0.5819,0.2934,-0.5584,-0.0916,-0.3010,-0.1537,0.2198,...,0.4805,0.4965,0.3680,0.8427,0.1042,0.1403,0.1758,1.2570,-0.5979,1.2250,-0.0553,0.7351,0.5810,0.9590,0.2427,0.0495,0.4141,0.8432,0.6162,-0.7318,1.2120,0.6362,-0.4427,0.1288,1.4840,0.1799,0.5367,-0.1111,-1.0120,0.6685,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,0.8507,-1.1520,-0.4201,-0.0958,0.4590,0.0803,0.2250,0.5293,0.2839,-0.3494,0.2883,0.9449,-0.1646,-0.2657,-0.3372,0.3135,-0.4316,0.4773,0.2075,-0.4216,-0.1161,-0.0499,-0.2627,0.9959,-0.2483,0.2655,-0.2102,...,0.4083,0.0319,0.3905,0.7099,0.2912,0.4151,-0.2840,-0.3104,-0.6373,0.2887,-0.0765,0.2539,0.4443,0.5932,0.2031,0.7639,0.5499,-0.3322,-0.0977,0.4329,-0.2782,0.7827,0.5934,0.3402,0.1499,0.4420,0.9366,0.8193,-0.4236,0.3192,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,1.2300,-0.4797,-0.5631,-0.0366,-1.8300,0.6057,-0.3278,0.6042,-0.3075,-0.1147,-0.0570,-0.0799,-0.8181,-1.5320,0.2307,0.4901,0.4780,-1.3970,4.6240,-0.0437,1.2870,-1.8530,0.6069,0.4290,0.1783,0.0018,-1.1800,...,-0.5477,-0.7576,-0.0444,0.1894,-0.0014,-2.3640,-0.4682,0.1210,-0.5177,-0.0604,0.1682,-0.4436,0.4963,0.1363,0.3335,0.9760,-0.0427,-0.1235,0.0959,0.0690,-0.9416,-0.7548,-0.1109,-0.6272,0.3019,0.1172,0.1093,-0.3113,0.3019,-0.0873,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,-1.0600,-0.8269,-0.3584,-0.8511,-0.5844,-2.5690,0.8183,-0.0532,-0.8554,0.1160,-2.3520,2.1200,-1.1580,-0.7191,-0.8004,-1.4670,-0.0107,-0.8995,0.2406,-0.2479,-1.0890,-0.7575,0.0881,-2.7370,0.8745,0.5787,-1.6740,...,-2.1220,-0.3752,-2.3820,-3.7350,-2.9740,-1.4930,-1.6600,-3.1660,0.2816,-0.2990,-1.1870,-0.5044,-1.7750,-1.6120,-0.9215,-1.0810,-3.0520,-3.4470,-2.7740,-1.8460,-0.5568,-3.3960,-2.9510,-1.1550,-3.2620,-1.5390,-2.4600,-0.9417,-1.5550,0.2431,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,0.8630,-0.2219,-0.5121,-0.9577,1.1750,0.2042,0.1970,0.1244,-1.7090,-0.3543,-0.5160,-0.3330,-0.2685,0.7649,0.2057,1.3720,0.6835,0.8056,-0.3754,-1.2090,0.2965,-0.0712,0.6389,0.6674,-0.0783,1.1740,-0.7110,...,-0.2274,0.3215,0.1535,-0.4640,-0.5943,0.3973,0.1500,0.5178,0.5159,0.6091,0.1813,-0.4249,0.7832,0.6529,0.5648,0.4817,0.0587,0.5303,0.6376,-0.3966,-1.4950,-0.9625,-0.0541,0.6273,0.4563,0.0698,0.8134,0.1924,0.6054,-0.1824,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [0]:
# get the dimensions of the data
(df_train.count() , len(df_train.columns))

Out[53]: (23814, 880)

In [0]:
# import sql function pyspark
import pyspark.sql.functions as f
# null values in each column
data_agg = df_train.agg(*[f.count(f.when(f.isnull(c), c)).alias(c) for c in df_train.columns])
data_agg.show()

+------+-------+-------+-------+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----

In [0]:
# no NA found
data_agg.toPandas()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,...,c-60,c-61,c-62,c-63,c-64,c-65,c-66,c-67,c-68,c-69,c-70,c-71,c-72,c-73,c-74,c-75,c-76,c-77,c-78,c-79,c-80,c-81,c-82,c-83,c-84,c-85,c-86,c-87,c-88,c-89,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_train.dtypes

Out[18]: [('sig_id', 'string'),
 ('cp_type', 'string'),
 ('cp_time', 'int'),
 ('cp_dose', 'string'),
 ('g-0', 'double'),
 ('g-1', 'double'),
 ('g-2', 'double'),
 ('g-3', 'double'),
 ('g-4', 'double'),
 ('g-5', 'double'),
 ('g-6', 'double'),
 ('g-7', 'double'),
 ('g-8', 'double'),
 ('g-9', 'double'),
 ('g-10', 'double'),
 ('g-11', 'double'),
 ('g-12', 'double'),
 ('g-13', 'double'),
 ('g-14', 'double'),
 ('g-15', 'double'),
 ('g-16', 'double'),
 ('g-17', 'double'),
 ('g-18', 'double'),
 ('g-19', 'double'),
 ('g-20', 'double'),
 ('g-21', 'double'),
 ('g-22', 'double'),
 ('g-23', 'double'),
 ('g-24', 'double'),
 ('g-25', 'double'),
 ('g-26', 'double'),
 ('g-27', 'double'),
 ('g-28', 'double'),
 ('g-29', 'double'),
 ('g-30', 'double'),
 ('g-31', 'double'),
 ('g-32', 'double'),
 ('g-33', 'double'),
 ('g-34', 'double'),
 ('g-35', 'double'),
 ('g-36', 'double'),
 ('g-37', 'double'),
 ('g-38', 'double'),
 ('g-39', 'double'),
 ('g-40', 'double'),
 ('g-41', 'double'),
 ('g-42', 'double'),
 ('g-43', 'double'),
 ('g-44', 'double'),
 ('g-45', 'double'),
 ('g-46', 'double'),
 ('g-47', 'double'),
 ('g-48', 'double'),
 ('g-49', 'double'),
 ('g-50', 'double'),
 ('g-51', 'double'),
 ('g-52', 'double'),
 ('g-53', 'double'),
 ('g-54', 'double'),
 ('g-55', 'double'),
 ('g-56', 'double'),
 ('g-57', 'double'),
 ('g-58', 'double'),
 ('g-59', 'double'),
 ('g-60', 'double'),
 ('g-61', 'double'),
 ('g-62', 'double'),
 ('g-63', 'double'),
 ('g-64', 'double'),
 ('g-65', 'double'),
 ('g-66', 'double'),
 ('g-67', 'double'),
 ('g-68', 'double'),
 ('g-69', 'double'),
 ('g-70', 'double'),
 ('g-71', 'double'),
 ('g-72', 'double'),
 ('g-73', 'double'),
 ('g-74', 'double'),
 ('g-75', 'double'),
 ('g-76', 'double'),
 ('g-77', 'double'),
 ('g-78', 'double'),
 ('g-79', 'double'),
 ('g-80', 'double'),
 ('g-81', 'double'),
 ('g-82', 'double'),
 ('g-83', 'double'),
 ('g-84', 'double'),
 ('g-85', 'double'),
 ('g-86', 'double'),
 ('g-87', 'double'),
 ('g-88', 'double'),
 ('g-89', 'double'),
 ('g-90', 'double'),
 ('g-91', 'double'),
 ('g-92', 'double'),
 ('g-93', 'double'),
 ('g-94', 'double'),
 ('g-95', 'double'),
 ('g-96', 'double'),
 ('g-97', 'double'),
 ('g-98', 'double'),
 ('g-99', 'double'),
 ('g-100', 'double'),
 ('g-101', 'double'),
 ('g-102', 'double'),
 ('g-103', 'double'),
 ('g-104', 'double'),
 ('g-105', 'double'),
 ('g-106', 'double'),
 ('g-107', 'double'),
 ('g-108', 'double'),
 ('g-109', 'double'),
 ('g-110', 'double'),
 ('g-111', 'double'),
 ('g-112', 'double'),
 ('g-113', 'double'),
 ('g-114', 'double'),
 ('g-115', 'double'),
 ('g-116', 'double'),
 ('g-117', 'double'),
 ('g-118', 'double'),
 ('g-119', 'double'),
 ('g-120', 'double'),
 ('g-121', 'double'),
 ('g-122', 'double'),
 ('g-123', 'double'),
 ('g-124', 'double'),
 ('g-125', 'double'),
 ('g-126', 'double'),
 ('g-127', 'double'),
 ('g-128', 'double'),
 ('g-129', 'double'),
 ('g-130', 'double'),
 ('g-131', 'double'),
 ('g-132', 'double'),
 ('g-133', 'double'),
 ('g-134', 'double'),
 ('g-135', 'double'),
 ('g-136', 'double'),
 ('g-137', 'double'),
 ('g-138', 'double'),
 ('g-139', 'double'),
 ('g-140', 'double'),
 ('g-141', 'double'),
 ('g-142', 'double'),
 ('g-143', 'double'),
 ('g-144', 'double'),
 ('g-145', 'double'),
 ('g-146', 'double'),
 ('g-147', 'double'),
 ('g-148', 'double'),
 ('g-149', 'double'),
 ('g-150', 'double'),
 ('g-151', 'double'),
 ('g-152', 'double'),
 ('g-153', 'double'),
 ('g-154', 'double'),
 ('g-155', 'double'),
 ('g-156', 'double'),
 ('g-157', 'double'),
 ('g-158', 'double'),
 ('g-159', 'double'),
 ('g-160', 'double'),
 ('g-161', 'double'),
 ('g-162', 'double'),
 ('g-163', 'double'),
 ('g-164', 'double'),
 ('g-165', 'double'),
 ('g-166', 'double'),
 ('g-167', 'double'),
 ('g-168', 'double'),
 ('g-169', 'double'),
 ('g-170', 'double'),
 ('g-171', 'double'),
 ('g-172', 'double'),
 ('g-173', 'double'),
 ('g-174', 'double'),
 ('g-175', 'double'),
 ('g-176', 'double'),
 ('g-177', 'double'),
 ('g-178', 'double'),
 ('g-179', 'double'),
 ('g-180', 'double'),
 ('g-181', 'double'),
 ('g-

In [0]:
categoricalColumns = [item[0] for item in df_train.dtypes if item[1].startswith('string') ]
categoricalColumns

Out[20]: ['sig_id', 'cp_type', 'cp_dose']

In [0]:
numericColumns = [item[0] for item in df_train.dtypes if item[1].startswith('double') ]
numericColumns

Out[21]: ['g-0',
 'g-1',
 'g-2',
 'g-3',
 'g-4',
 'g-5',
 'g-6',
 'g-7',
 'g-8',
 'g-9',
 'g-10',
 'g-11',
 'g-12',
 'g-13',
 'g-14',
 'g-15',
 'g-16',
 'g-17',
 'g-18',
 'g-19',
 'g-20',
 'g-21',
 'g-22',
 'g-23',
 'g-24',
 'g-25',
 'g-26',
 'g-27',
 'g-28',
 'g-29',
 'g-30',
 'g-31',
 'g-32',
 'g-33',
 'g-34',
 'g-35',
 'g-36',
 'g-37',
 'g-38',
 'g-39',
 'g-40',
 'g-41',
 'g-42',
 'g-43',
 'g-44',
 'g-45',
 'g-46',
 'g-47',
 'g-48',
 'g-49',
 'g-50',
 'g-51',
 'g-52',
 'g-53',
 'g-54',
 'g-55',
 'g-56',
 'g-57',
 'g-58',
 'g-59',
 'g-60',
 'g-61',
 'g-62',
 'g-63',
 'g-64',
 'g-65',
 'g-66',
 'g-67',
 'g-68',
 'g-69',
 'g-70',
 'g-71',
 'g-72',
 'g-73',
 'g-74',
 'g-75',
 'g-76',
 'g-77',
 'g-78',
 'g-79',
 'g-80',
 'g-81',
 'g-82',
 'g-83',
 'g-84',
 'g-85',
 'g-86',
 'g-87',
 'g-88',
 'g-89',
 'g-90',
 'g-91',
 'g-92',
 'g-93',
 'g-94',
 'g-95',
 'g-96',
 'g-97',
 'g-98',
 'g-99',
 'g-100',
 'g-101',
 'g-102',
 'g-103',
 'g-104',
 'g-105',
 'g-106',
 'g-107',
 'g-108',
 'g-109',
 'g-110',
 'g-111',
 'g-112',
 'g-113',
 'g-114',
 'g-115',
 'g-116',
 'g-117',
 'g-118',
 'g-119',
 'g-120',
 'g-121',
 'g-122',
 'g-123',
 'g-124',
 'g-125',
 'g-126',
 'g-127',
 'g-128',
 'g-129',
 'g-130',
 'g-131',
 'g-132',
 'g-133',
 'g-134',
 'g-135',
 'g-136',
 'g-137',
 'g-138',
 'g-139',
 'g-140',
 'g-141',
 'g-142',
 'g-143',
 'g-144',
 'g-145',
 'g-146',
 'g-147',
 'g-148',
 'g-149',
 'g-150',
 'g-151',
 'g-152',
 'g-153',
 'g-154',
 'g-155',
 'g-156',
 'g-157',
 'g-158',
 'g-159',
 'g-160',
 'g-161',
 'g-162',
 'g-163',
 'g-164',
 'g-165',
 'g-166',
 'g-167',
 'g-168',
 'g-169',
 'g-170',
 'g-171',
 'g-172',
 'g-173',
 'g-174',
 'g-175',
 'g-176',
 'g-177',
 'g-178',
 'g-179',
 'g-180',
 'g-181',
 'g-182',
 'g-183',
 'g-184',
 'g-185',
 'g-186',
 'g-187',
 'g-188',
 'g-189',
 'g-190',
 'g-191',
 'g-192',
 'g-193',
 'g-194',
 'g-195',
 'g-196',
 'g-197',
 'g-198',
 'g-199',
 'g-200',
 'g-201',
 'g-202',
 'g-203',
 'g-204',
 'g-205',
 'g-206',
 'g-207',
 'g-208',
 'g-209',
 'g-210',
 'g-211',
 'g-212',
 'g-213',
 'g-214',
 'g-215',
 'g-216',
 'g-217',
 'g-218',
 'g-219',
 'g-220',
 'g-221',
 'g-222',
 'g-223',
 'g-224',
 'g-225',
 'g-226',
 'g-227',
 'g-228',
 'g-229',
 'g-230',
 'g-231',
 'g-232',
 'g-233',
 'g-234',
 'g-235',
 'g-236',
 'g-237',
 'g-238',
 'g-239',
 'g-240',
 'g-241',
 'g-242',
 'g-243',
 'g-244',
 'g-245',
 'g-246',
 'g-247',
 'g-248',
 'g-249',
 'g-250',
 'g-251',
 'g-252',
 'g-253',
 'g-254',
 'g-255',
 'g-256',
 'g-257',
 'g-258',
 'g-259',
 'g-260',
 'g-261',
 'g-262',
 'g-263',
 'g-264',
 'g-265',
 'g-266',
 'g-267',
 'g-268',
 'g-269',
 'g-270',
 'g-271',
 'g-272',
 'g-273',
 'g-274',
 'g-275',
 'g-276',
 'g-277',
 'g-278',
 'g-279',
 'g-280',
 'g-281',
 'g-282',
 'g-283',
 'g-284',
 'g-285',
 'g-286',
 'g-287',
 'g-288',
 'g-289',
 'g-290',
 'g-291',
 'g-292',
 'g-293',
 'g-294',
 'g-295',
 'g-296',
 'g-297',
 'g-298',
 'g-299',
 'g-300',
 'g-301',
 'g-302',
 'g-303',
 'g-304',
 'g-305',
 'g-306',
 'g-307',
 'g-308',
 'g-309',
 'g-310',
 'g-311',
 'g-312',
 'g-313',
 'g-314',
 'g-315',
 'g-316',
 'g-317',
 'g-318',
 'g-319',
 'g-320',
 'g-321',
 'g-322',
 'g-323',
 'g-324',
 'g-325',
 'g-326',
 'g-327',
 'g-328',
 'g-329',
 'g-330',
 'g-331',
 'g-332',
 'g-333',
 'g-334',
 'g-335',
 'g-336',
 'g-337',
 'g-338',
 'g-339',
 'g-340',
 'g-341',
 'g-342',
 'g-343',
 'g-344',
 'g-345',
 'g-346',
 'g-347',
 'g-348',
 'g-349',
 'g-350',
 'g-351',
 'g-352',
 'g-353',
 'g-354',
 'g-355',
 'g-356',
 'g-357',
 'g-358',
 'g-359',
 'g-360',
 'g-361',
 'g-362',
 'g-363',
 'g-364',
 'g-365',
 'g-366',
 'g-367',
 'g-368',
 'g-369',
 'g-370',
 'g-371',
 'g-372',
 'g-373',
 'g-374',
 'g-375',
 'g-376',
 'g-377',
 'g-378',
 'g-379',
 'g-380',
 'g-381',
 'g-382',
 'g-383',
 'g-384',
 'g-385',
 'g-386',
 'g-387',
 'g-388',
 'g-389',
 'g-390',
 'g-391',
 'g-392',
 'g-393',
 'g-394',
 'g-395',
 'g-396',
 'g-397',
 'g-398',
 'g-399',
 'g-400',
 'g-401',
 'g-402',
 'g-403',
 'g-404',
 'g-405',
 'g-406',
 'g-407',
 'g-408',
 'g-409',
 

In [0]:
#Preprocessing steps
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# create object of StringIndexer class and specify input and output column
SI_type = StringIndexer(inputCol='cp_type',outputCol='cp_type_Index')
SI_dose = StringIndexer(inputCol='cp_dose',outputCol='cp_dose_Index')

# transform the data
df_train = SI_type.fit(df_train).transform(df_train)
df_train = SI_dose.fit(df_train).transform(df_train)


In [0]:
# view the transformed data
df_train.select('cp_type', 'cp_type_Index', 'cp_dose', 'cp_dose_Index').show(10)

+-------+-------------+-------+-------------+
cp_type|cp_type_Index|cp_dose|cp_dose_Index|
+-------+-------------+-------+-------------+
 trt_cp| 0.0| D1| 0.0|
 trt_cp| 0.0| D1| 0.0|
 trt_cp| 0.0| D1| 0.0|
 trt_cp| 0.0| D1| 0.0|
 trt_cp| 0.0| D2| 1.0|
 trt_cp| 0.0| D1| 0.0|
 trt_cp| 0.0| D2| 1.0|
 trt_cp| 0.0| D1| 0.0|
 trt_cp| 0.0| D1| 0.0|
 trt_cp| 0.0| D2| 1.0|
+-------+-------------+-------+-------------+
only showing top 10 rows

In [0]:
# create object and specify input and output column
OHE = OneHotEncoder(inputCols=['cp_type_Index','cp_dose_Index'],outputCols=['cp_type_OHE', 'cp_dose_OHE'])

# transform the data
df_train = OHE.fit(df_train).transform(df_train)

In [0]:
# view and transform the data
df_train.select('cp_type', 'cp_type_Index', 'cp_type_OHE','cp_dose','cp_dose_Index','cp_dose_OHE').show(10)

+-------+-------------+-------------+-------+-------------+-------------+
cp_type|cp_type_Index| cp_type_OHE|cp_dose|cp_dose_Index| cp_dose_OHE|
+-------+-------------+-------------+-------+-------------+-------------+
 trt_cp| 0.0|(1,[0],[1.0])| D1| 0.0|(1,[0],[1.0])|
 trt_cp| 0.0|(1,[0],[1.0])| D1| 0.0|(1,[0],[1.0])|
 trt_cp| 0.0|(1,[0],[1.0])| D1| 0.0|(1,[0],[1.0])|
 trt_cp| 0.0|(1,[0],[1.0])| D1| 0.0|(1,[0],[1.0])|
 trt_cp| 0.0|(1,[0],[1.0])| D2| 1.0| (1,[],[])|
 trt_cp| 0.0|(1,[0],[1.0])| D1| 0.0|(1,[0],[1.0])|
 trt_cp| 0.0|(1,[0],[1.0])| D2| 1.0| (1,[],[])|
 trt_cp| 0.0|(1,[0],[1.0])| D1| 0.0|(1,[0],[1.0])|
 trt_cp| 0.0|(1,[0],[1.0])| D1| 0.0|(1,[0],[1.0])|
 trt_cp| 0.0|(1,[0],[1.0])| D2| 1.0| (1,[],[])|
+-------+-------------+-------------+-------+-------------+-------------+
only showing top 10 rows

In [0]:
df_train.dtypes

Out[28]: [('sig_id', 'string'),
 ('cp_type', 'string'),
 ('cp_time', 'int'),
 ('cp_dose', 'string'),
 ('g-0', 'double'),
 ('g-1', 'double'),
 ('g-2', 'double'),
 ('g-3', 'double'),
 ('g-4', 'double'),
 ('g-5', 'double'),
 ('g-6', 'double'),
 ('g-7', 'double'),
 ('g-8', 'double'),
 ('g-9', 'double'),
 ('g-10', 'double'),
 ('g-11', 'double'),
 ('g-12', 'double'),
 ('g-13', 'double'),
 ('g-14', 'double'),
 ('g-15', 'double'),
 ('g-16', 'double'),
 ('g-17', 'double'),
 ('g-18', 'double'),
 ('g-19', 'double'),
 ('g-20', 'double'),
 ('g-21', 'double'),
 ('g-22', 'double'),
 ('g-23', 'double'),
 ('g-24', 'double'),
 ('g-25', 'double'),
 ('g-26', 'double'),
 ('g-27', 'double'),
 ('g-28', 'double'),
 ('g-29', 'double'),
 ('g-30', 'double'),
 ('g-31', 'double'),
 ('g-32', 'double'),
 ('g-33', 'double'),
 ('g-34', 'double'),
 ('g-35', 'double'),
 ('g-36', 'double'),
 ('g-37', 'double'),
 ('g-38', 'double'),
 ('g-39', 'double'),
 ('g-40', 'double'),
 ('g-41', 'double'),
 ('g-42', 'double'),
 ('g-43', 'double'),
 ('g-44', 'double'),
 ('g-45', 'double'),
 ('g-46', 'double'),
 ('g-47', 'double'),
 ('g-48', 'double'),
 ('g-49', 'double'),
 ('g-50', 'double'),
 ('g-51', 'double'),
 ('g-52', 'double'),
 ('g-53', 'double'),
 ('g-54', 'double'),
 ('g-55', 'double'),
 ('g-56', 'double'),
 ('g-57', 'double'),
 ('g-58', 'double'),
 ('g-59', 'double'),
 ('g-60', 'double'),
 ('g-61', 'double'),
 ('g-62', 'double'),
 ('g-63', 'double'),
 ('g-64', 'double'),
 ('g-65', 'double'),
 ('g-66', 'double'),
 ('g-67', 'double'),
 ('g-68', 'double'),
 ('g-69', 'double'),
 ('g-70', 'double'),
 ('g-71', 'double'),
 ('g-72', 'double'),
 ('g-73', 'double'),
 ('g-74', 'double'),
 ('g-75', 'double'),
 ('g-76', 'double'),
 ('g-77', 'double'),
 ('g-78', 'double'),
 ('g-79', 'double'),
 ('g-80', 'double'),
 ('g-81', 'double'),
 ('g-82', 'double'),
 ('g-83', 'double'),
 ('g-84', 'double'),
 ('g-85', 'double'),
 ('g-86', 'double'),
 ('g-87', 'double'),
 ('g-88', 'double'),
 ('g-89', 'double'),
 ('g-90', 'double'),
 ('g-91', 'double'),
 ('g-92', 'double'),
 ('g-93', 'double'),
 ('g-94', 'double'),
 ('g-95', 'double'),
 ('g-96', 'double'),
 ('g-97', 'double'),
 ('g-98', 'double'),
 ('g-99', 'double'),
 ('g-100', 'double'),
 ('g-101', 'double'),
 ('g-102', 'double'),
 ('g-103', 'double'),
 ('g-104', 'double'),
 ('g-105', 'double'),
 ('g-106', 'double'),
 ('g-107', 'double'),
 ('g-108', 'double'),
 ('g-109', 'double'),
 ('g-110', 'double'),
 ('g-111', 'double'),
 ('g-112', 'double'),
 ('g-113', 'double'),
 ('g-114', 'double'),
 ('g-115', 'double'),
 ('g-116', 'double'),
 ('g-117', 'double'),
 ('g-118', 'double'),
 ('g-119', 'double'),
 ('g-120', 'double'),
 ('g-121', 'double'),
 ('g-122', 'double'),
 ('g-123', 'double'),
 ('g-124', 'double'),
 ('g-125', 'double'),
 ('g-126', 'double'),
 ('g-127', 'double'),
 ('g-128', 'double'),
 ('g-129', 'double'),
 ('g-130', 'double'),
 ('g-131', 'double'),
 ('g-132', 'double'),
 ('g-133', 'double'),
 ('g-134', 'double'),
 ('g-135', 'double'),
 ('g-136', 'double'),
 ('g-137', 'double'),
 ('g-138', 'double'),
 ('g-139', 'double'),
 ('g-140', 'double'),
 ('g-141', 'double'),
 ('g-142', 'double'),
 ('g-143', 'double'),
 ('g-144', 'double'),
 ('g-145', 'double'),
 ('g-146', 'double'),
 ('g-147', 'double'),
 ('g-148', 'double'),
 ('g-149', 'double'),
 ('g-150', 'double'),
 ('g-151', 'double'),
 ('g-152', 'double'),
 ('g-153', 'double'),
 ('g-154', 'double'),
 ('g-155', 'double'),
 ('g-156', 'double'),
 ('g-157', 'double'),
 ('g-158', 'double'),
 ('g-159', 'double'),
 ('g-160', 'double'),
 ('g-161', 'double'),
 ('g-162', 'double'),
 ('g-163', 'double'),
 ('g-164', 'double'),
 ('g-165', 'double'),
 ('g-166', 'double'),
 ('g-167', 'double'),
 ('g-168', 'double'),
 ('g-169', 'double'),
 ('g-170', 'double'),
 ('g-171', 'double'),
 ('g-172', 'double'),
 ('g-173', 'double'),
 ('g-174', 'double'),
 ('g-175', 'double'),
 ('g-176', 'double'),
 ('g-177', 'double'),
 ('g-178', 'double'),
 ('g-179', 'double'),
 ('g-180', 'double'),
 ('g-181', 'double'),
 ('g-

In [0]:
from pyspark.ml.feature import VectorAssembler

df1 = df_train.drop('sig_id','cp_type','cp_dose')

# specify the input and output columns of the vector assembler
assembler = VectorAssembler(inputCols= df1.columns,
                           outputCol='features')

# transform the data
final_df_train = assembler.transform(df_train)

In [0]:
final_df_train.select('features').show()

+--------------------+
 features|
+--------------------+
[24.0,1.062,0.557...|
[72.0,0.0743,0.40...|
[48.0,0.628,0.581...|
[48.0,-0.5138,-0....|
[72.0,-0.3254,-0....|
[24.0,-0.6111,0.2...|
[24.0,2.044,1.7,-...|
[48.0,0.2711,0.51...|
[48.0,-0.3014,0.5...|
[48.0,-0.063,0.25...|
[72.0,-0.2875,0.0...|
[48.0,-0.3864,-0....|
[48.0,0.003,0.718...|
[48.0,-0.6884,-0....|
[72.0,0.4242,1.70...|
[48.0,0.0667,-0.6...|
[48.0,8.738,0.191...|
[24.0,-0.4764,-0....|
[72.0,-0.4694,-1....|
[48.0,-0.1428,-0....|
+--------------------+
only showing top 20 rows

In [0]:
df_target.toPandas().head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,...,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_target.count(),len(df_target.columns)

Out[60]: (23814, 207)

In [0]:
df2 = df_target.drop('sig_id')

# specify the input and output columns of the vector assembler
assembler = VectorAssembler(inputCols= df2.columns,
                           outputCol='labels')

# transform the data
final_df_target = assembler.transform(df_target)

In [0]:
final_df_target.select('labels').show()

+--------------------+
 labels|
+--------------------+
 (206,[101],[1.0])|
 (206,[],[])|
(206,[48,119,149]...|
 (206,[],[])|
 (206,[54],[1.0])|
 (206,[101],[1.0])|
 (206,[],[])|
 (206,[63],[1.0])|
 (206,[],[])|
 (206,[150],[1.0])|
 (206,[173],[1.0])|
(206,[79,93],[1.0...|
 (206,[135],[1.0])|
 (206,[77],[1.0])|
 (206,[],[])|
 (206,[162],[1.0])|
(206,[136,163],[1...|
 (206,[103],[1.0])|
(206,[38,48,89,11...|
 (206,[77],[1.0])|
+--------------------+
only showing top 20 rows

In [0]:
final_df = final_df_train.join(final_df_target.alias('target'), ['sig_id'])

In [0]:
model_df = final_df.select(['features','labels'])
model_df.printSchema()

root
-- features: vector (nullable = true)
-- labels: vector (nullable = true)

In [0]:
#Split into training & validation Dataframe
training_df,valid_df = model_df.randomSplit([0.75,0.25])

In [0]:
#Create a logistic regression model object
from pyspark.ml.classification import LogisticRegression
log_reg=LogisticRegression().fit(training_df)

--------------------------------------------------------------------------- 
 IllegalArgumentException Traceback (most recent call last)
 <command-3222623531072962> in <module> 
 1 #Create a logistic regression model object 
 2 from pyspark . ml . classification import LogisticRegression
 ----> 3 log_reg = LogisticRegression ( ) . fit ( training_df ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 127 return self . copy ( params ) . _fit ( dataset ) 
 128 else : 
 --> 129 return self . _fit ( dataset ) 
 130 else : 
 131 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 319 
 320 def _fit ( self , dataset ) : 
 --> 321 java_model = self . _fit_java ( dataset ) 
 322 model = self . _create_model ( java_model ) 
 323 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 316 """
 317 self . _transfer_params_to_java ( ) 
 --> 318 return self . _java_obj . fit ( dataset . _jdf ) 
 319 
 320 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 IllegalArgumentException : label does not exist. Available: features, labels